In [43]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [44]:
final_data = []
year = 2017
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

for week in weeks:
    url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    week_data = []
    for board in soup.select('.ScoreboardScoreCell'):
        title = board.find_previous(class_='Card__Header__Title').text
        teams = [t.text for t in board.select('.ScoreCell__TeamName')]
        scores = [s.text for s in board.select('.ScoreCell__Score')] or ['-', '-']

        week_data.append((week, title, teams[0], scores[0], teams[1], scores[1]))

    final_data.extend(week_data)

df_scores_2017 = pd.DataFrame(final_data, columns=['Week', 'Date', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
df_scores_2017 = df_scores_2017.rename(columns={'Team 1': 'Away Team', 'Score 1': 'Away Score', 'Team 2': 'Home Team', 'Score 2':'Home Score'})
df_scores_2017['Final Score'] = df_scores_2017['Away Score'] + ' - ' + df_scores_2017['Home Score']
#print(df.to_markdown(index=False))
# Replace missing scores with NaN and dtype convert object to int 
df_scores_2017['Away Score'] = df_scores_2017['Away Score'].replace('-', np.nan)
df_scores_2017['Home Score'] = df_scores_2017['Home Score'].replace('-', np.nan)
df_scores_2017['Away Score'] = pd.to_numeric(df_scores_2017['Away Score'])
df_scores_2017['Home Score'] =  pd.to_numeric(df_scores_2017['Home Score'])

# Calculate total points
df_scores_2017['Total Points'] = df_scores_2017['Away Score'] + df_scores_2017['Home Score']

# Calculate winner
df_scores_2017['Winner'] = df_scores_2017.apply(lambda row: row['Away Team'] if row['Away Score'] > row['Home Score'] else row['Home Team'], axis=1)

# Calculate score differential
df_scores_2017['Away Score Differential'] = df_scores_2017['Home Score'] - df_scores_2017['Away Score']
df_scores_2017['Home Score Differential'] = df_scores_2017['Away Score'] - df_scores_2017['Home Score']
df_scores_2017['Matchup'] = df_scores_2017['Away Team'] + ' @ ' + df_scores_2017['Home Team']
df_scores_2017['Date'] = pd.to_datetime(df_scores_2017['Date'], format='%A, %B %d, %Y').dt.strftime('%Y-%m-%d')
# Use a lambda function to create the 'Home_Away_Winner' column
df_scores_2017['Home_Away_Winner'] = df_scores_2017.apply(lambda row: 'Home' 
                                                          if row['Winner'] == row['Home Team'] 
                                                          else 'Away' if row['Winner'] == row['Away Team'] else 'No Winner', axis=1)
df_scores_2017.tail()

,Week,Date,Away Team,Away Score,Home Team,Home Score,Final Score,Total Points,Winner,Away Score Differential,Home Score Differential,Matchup,Home_Away_Winner
252,17,2017-12-31,Bills,22.0,Dolphins,16.0,22 - 16,38.0,Bills,-6.0,6.0,Bills @ Dolphins,Away
253,17,2017-12-31,Raiders,10.0,Chargers,30.0,10 - 30,40.0,Chargers,20.0,-20.0,Raiders @ Chargers,Home
254,17,2017-12-31,Cardinals,26.0,Seahawks,24.0,26 - 24,50.0,Cardinals,-2.0,2.0,Cardinals @ Seahawks,Away
255,17,2017-12-31,Saints,24.0,Buccaneers,31.0,24 - 31,55.0,Buccaneers,7.0,-7.0,Saints @ Buccaneers,Home
256,17,2017-12-31,Bengals,31.0,Ravens,27.0,31 - 27,58.0,Bengals,-4.0,4.0,Bengals @ Ravens,Away


In [45]:
#import consensus data for spreads 
game_dates = list(df_scores_2017['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

spreads = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        spreads.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_spreads_2017 = pd.concat(spreads, ignore_index=True)

In [46]:
#clean matchup column 
df_spreads_2017['Matchup'] = df_spreads_2017['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_spreads_2017[['Away Team', 'Home Team']] = df_spreads_2017['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_spreads_2017['Away Team'] = df_spreads_2017['Away Team'].str.strip()
df_spreads_2017['Home Team'] = df_spreads_2017['Home Team'].str.strip()
#split to create away and home spread cols 
df_spreads_2017[['Away Spread', 'Home Spread']] = df_spreads_2017['Sides'].str.split(expand=True)
#split to create away and home consensus cols 
df_spreads_2017[['Away Consensus', 'Home Consensus']] = df_spreads_2017['Consensus'].str.split(expand=True)
# convert to dtype = float for consensus cols 
df_spreads_2017['Away Consensus'] =  df_spreads_2017['Away Consensus'].str.rstrip('%').astype(int)/100
df_spreads_2017['Home Consensus'] = df_spreads_2017['Home Consensus'].str.rstrip('%').astype(int)/100
#create consensus favorite vs. underdog cols 
df_spreads_2017['Con. Spread Favorite'] = df_spreads_2017.apply(lambda x: x['Away Team'] 
                                                                if x['Away Consensus'] > x['Home Consensus'] else x['Home Team'], axis=1)
df_spreads_2017['Con. Spread Underdog'] = df_spreads_2017.apply(lambda x: x['Away Team'] 
                                                                if x['Away Consensus'] < x['Home Consensus'] else x['Home Team'], axis=1)
# Define the dictionary mapping for replacements
replacement_dict = {'LV': 'Raiders',
                    'KC': 'Chiefs',
                    'NE': 'Patriots',
                    'GB': 'Packers',
                    'NO': 'Saints',
                    'BAL': 'Ravens',
                    'SF': '49ers',
                    'NYG': 'Giants',
                    'JAC': 'Jaguars',
                    'PIT': 'Steelers',
                    'TB': 'Buccaneers',
                    'CLE': 'Browns',
                    'IND': 'Colts',
                    'PHI': 'Eagles',
                    'ARI': 'Cardinals',
                    'WAS': 'Commanders',
                    'ATL': 'Falcons',
                    'NYJ': 'Jets',
                    'SEA': 'Seahawks',
                    'MIA': 'Dolphins',
                    'HOU': 'Texans',
                    'CAR': 'Panthers',
                    'CHI': 'Bears',
                    'CIN': 'Bengals',
                    'LAR': 'Rams',
                    'BUF': 'Bills',
                    'DET': 'Lions',
                    'DEN': 'Broncos',
                    'TEN': 'Titans',
                    'MIN': 'Vikings',
                    'LAC': 'Chargers',
                    'DAL': 'Cowboys'}
# Replace the values in the DataFrame column using map()
df_spreads_2017['Away Team'] = df_spreads_2017['Away Team'].map(replacement_dict)
df_spreads_2017['Home Team'] = df_spreads_2017['Home Team'].map(replacement_dict)
df_spreads_2017['Con. Spread Favorite'] = df_spreads_2017['Con. Spread Favorite'].map(replacement_dict)
df_spreads_2017['Con. Spread Underdog'] = df_spreads_2017['Con. Spread Underdog'].map(replacement_dict)
df_spreads_2017['Matchup'] = df_spreads_2017['Away Team'] + ' @ ' + df_spreads_2017['Home Team']
df_spreads_2017.tail()

,Matchup,Date,Consensus,Sides,Picks,Indepth,Away Team,Home Team,Away Spread,Home Spread,Away Consensus,Home Consensus,Con. Spread Favorite,Con. Spread Underdog
252,Bears @ Vikings,2017-12-31,53% 47%,+13 -13,901 796,Details,Bears,Vikings,+13,-13,0.53,0.47,Bears,Vikings
253,Cardinals @ Seahawks,2017-12-31,52% 48%,+8 -8,925 866,Details,Cardinals,Seahawks,+8,-8,0.52,0.48,Cardinals,Seahawks
254,49ers @ Rams,2017-12-31,50% 50%,-6 +6,877 894,Details,49ers,Rams,-6,+6,0.50,0.50,Rams,Rams
255,Cowboys @ Eagles,2017-12-31,50% 50%,-3.5 +3.5,837 850,Details,Cowboys,Eagles,-3.5,+3.5,0.50,0.50,Eagles,Eagles
256,Raiders @ Chargers,2017-12-31,50% 50%,+7 -7,867 872,Details,Raiders,Chargers,+7,-7,0.50,0.50,Chargers,Chargers


In [47]:
#import consensus data for spreads 
game_dates = list(df_scores_2017['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

totals = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        totals.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_totals_2017 = pd.concat(totals, ignore_index=True)
df_totals_2017.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth
252,NFL No Tb,2017-12-31,55 % Under 45 % Over,49.0,624 513,Details
253,NFL Kc Den,2017-12-31,53 % Over 47 % Under,37.5,658 583,Details
254,NFL Cin Bal,2017-12-31,52 % Under 48 % Over,40.0,597 544,Details
255,NFL Cle Pit,2017-12-31,51 % Over 49 % Under,38.0,611 592,Details
256,NFL Jac Ten,2017-12-31,51 % Under 49 % Over,40.0,609 591,Details


In [48]:
#clean matchup column 
df_totals_2017['Matchup'] = df_totals_2017['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_totals_2017[['Away Team', 'Home Team']] = df_totals_2017['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_totals_2017['Away Team'] = df_totals_2017['Away Team'].str.strip()
df_totals_2017['Home Team'] = df_totals_2017['Home Team'].str.strip()
# create consensus columns 
df_totals_2017['Over Consensus'] = df_totals_2017['Consensus'].str.extract(r'(\d+) % Over').astype(int)/100
df_totals_2017['Under Consensus'] = df_totals_2017['Consensus'].str.extract(r'(\d+) % Under').astype(int)/100
#create consensus favorites vs underdogs 
df_totals_2017['Con. Total Favorite'] = df_totals_2017[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] > x['Under Consensus'] else 'Under', axis=1)
df_totals_2017['Con. Total Underdog'] = df_totals_2017[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] < x['Under Consensus'] else 'Under', axis=1)
# Replace the values in the DataFrame column using map() from our previously defined replacement_dict
df_totals_2017['Away Team'] = df_totals_2017['Away Team'].map(replacement_dict)
df_totals_2017['Home Team'] = df_totals_2017['Home Team'].map(replacement_dict)
df_totals_2017['Matchup'] = df_totals_2017['Away Team'] + ' @ ' + df_totals_2017['Home Team']
df_totals_2017.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth,Away Team,Home Team,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog
252,Saints @ Buccaneers,2017-12-31,55 % Under 45 % Over,49.0,624 513,Details,Saints,Buccaneers,0.45,0.55,Under,Over
253,Chiefs @ Broncos,2017-12-31,53 % Over 47 % Under,37.5,658 583,Details,Chiefs,Broncos,0.53,0.47,Over,Under
254,Bengals @ Ravens,2017-12-31,52 % Under 48 % Over,40.0,597 544,Details,Bengals,Ravens,0.48,0.52,Under,Over
255,Browns @ Steelers,2017-12-31,51 % Over 49 % Under,38.0,611 592,Details,Browns,Steelers,0.51,0.49,Over,Under
256,Jaguars @ Titans,2017-12-31,51 % Under 49 % Over,40.0,609 591,Details,Jaguars,Titans,0.49,0.51,Under,Over


In [49]:
print(df_scores_2017.columns)
print(df_spreads_2017.columns)
print(df_totals_2017.columns)

Index(['Week', 'Date', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Final Score', 'Total Points', 'Winner', 'Away Score Differential',
       'Home Score Differential', 'Matchup', 'Home_Away_Winner'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Sides', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Total', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog'],
      dtype='object')


In [50]:
# pd.merge(df1, df2, on=['Key_Column_1', 'Key_Column_2'], how='inner')
combined_df = pd.merge(df_scores_2017[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 
                                       'Winner', 'Home_Away_Winner', 'Away Team', 'Away Score', 
                                       'Away Score Differential', 'Home Team', 'Home Score', 'Home Score Differential']], 
                       df_spreads_2017[['Matchup', 'Date', 'Away Team','Away Spread', 'Home Team', 
                                        'Home Spread', 'Away Consensus', 'Home Consensus', 
                                        'Con. Spread Favorite','Con. Spread Underdog']],
                       on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                       how='inner')

In [51]:
final_df_nfl_2017 = pd.merge(combined_df[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
                                           'Home_Away_Winner', 'Away Team', 'Away Score',
                                           'Away Score Differential', 'Home Team', 'Home Score',
                                           'Home Score Differential', 'Away Spread', 'Home Spread',
                                           'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
                                           'Con. Spread Underdog']],
                            df_totals_2017[['Matchup', 'Date', 'Away Team', 'Home Team', 'Total',
                                            'Over Consensus', 'Under Consensus',
                                            'Con. Total Favorite', 'Con. Total Underdog']],
                             on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                             how='inner')

final_df_nfl_2017['Win Marginal'] = np.abs(final_df_nfl_2017['Away Score'] - final_df_nfl_2017['Home Score'])

In [52]:
# 1) did away vs home team cover? -- Done 
# Convert 'Away Spread' and 'Home Spread' columns to numeric values
final_df_nfl_2017['Away Spread'] = pd.to_numeric(final_df_nfl_2017['Away Spread'])
final_df_nfl_2017['Home Spread'] = pd.to_numeric(final_df_nfl_2017['Home Spread'])

final_df_nfl_2017['Spread Covering Team'] = final_df_nfl_2017.apply(lambda row: row['Home Team'] 
                                                              if row['Home Score'] + row['Home Spread'] >= row['Away Score'] 
                                                              else row['Away Team'], axis=1)
final_df_nfl_2017[['Matchup', 'Final Score','Home Team', 'Home Spread', 'Home Score', 'Home Spread','Away Team', 'Away Spread', 'Spread Covering Team']].head()

,Matchup,Final Score,Home Team,Home Spread,Home Score,Home Spread,Away Team,Away Spread,Spread Covering Team
0,Chiefs @ Patriots,42 - 27,Patriots,-8.0,27.0,-8.0,Chiefs,8.0,Chiefs
1,Jets @ Bills,12 - 21,Bills,-7.5,21.0,-7.5,Jets,7.5,Bills
2,Falcons @ Bears,23 - 17,Bears,6.5,17.0,6.5,Falcons,-6.5,Bears
3,Ravens @ Bengals,20 - 0,Bengals,-3.0,0.0,-3.0,Ravens,3.0,Ravens
4,Steelers @ Browns,21 - 18,Browns,9.5,18.0,9.5,Steelers,-9.5,Browns


In [53]:
# 2) Did over/under hit? -- Done

# Create "Totals Covered" column
final_df_nfl_2017['Totals Covering Line'] = final_df_nfl_2017.apply(lambda row: 'Over' 
                                          if row['Total Points'] > row['Total'] 
                                          else ('Under' if row['Total Points'] < row['Total'] else 'Push'), axis=1)

# Print the updated DataFrame
final_df_nfl_2017[['Matchup', 'Total Points', 'Total', 'Totals Covering Line']].head()

,Matchup,Total Points,Total,Totals Covering Line
0,Chiefs @ Patriots,69.0,47.5,Over
1,Jets @ Bills,33.0,42.0,Under
2,Falcons @ Bears,40.0,48.0,Under
3,Ravens @ Bengals,20.0,41.5,Under
4,Steelers @ Browns,39.0,47.0,Under


In [54]:
# 3) Did consensus fav vs underdog win? [Spread] -- Done

# Create "Consensus Spread Winner" column
final_df_nfl_2017['Consensus Spread Winner'] = final_df_nfl_2017.apply(lambda row: 'Majority' 
                                            if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] > 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] > 0.5))
                                            else 'Minority' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] < 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] < 0.5))
                                            else 'Push' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] == 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] == 0.5))
                                            else '', axis=1)

# Print the updated DataFrame
final_df_nfl_2017[['Away Team', 'Home Team', 'Away Consensus', 'Home Consensus', 'Consensus Spread Winner']].head()

,Away Team,Home Team,Away Consensus,Home Consensus,Consensus Spread Winner
0,Chiefs,Patriots,0.52,0.48,Majority
1,Jets,Bills,0.53,0.47,Minority
2,Falcons,Bears,0.71,0.29,Minority
3,Ravens,Bengals,0.35,0.65,Minority
4,Steelers,Browns,0.62,0.38,Minority


In [55]:
# 4) Did consensus fav vs underdog win? [Totals] -- Done
# Create "Total Consensus Winner" column
final_df_nfl_2017['Total Consensus Winner'] = final_df_nfl_2017.apply(lambda row: 'Majority' 
                                                  if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] > 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] > 0.5))
                                                  else 'Minority' if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] < 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] < 0.5))
                                                  else 'Push' if (row['Under Consensus'] == 0.5) or (row['Over Consensus'] == 0.5)
                                                  else '', axis=1)

# Print the updated DataFrame
#final_df_nfl_2017[['Matchup', 'Total Points', 'Total', 'Totals Covering Line', 'Over Consensus', 'Away Consensus', 'Total Consensus Winner']]

In [56]:
final_df_nfl_2017.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner'],
      dtype='object')

In [57]:
final_df_nfl_2017

,Week,Date,Matchup,Final Score,Total Points,Winner,Home_Away_Winner,Away Team,Away Score,Away Score Differential,...,Total,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog,Win Marginal,Spread Covering Team,Totals Covering Line,Consensus Spread Winner,Total Consensus Winner
0,1,2017-09-07,Chiefs @ Patriots,42 - 27,69.0,Chiefs,Away,Chiefs,42.0,-15.0,...,47.5,0.56,0.44,Over,Under,15.0,Chiefs,Over,Majority,Majority
1,1,2017-09-10,Jets @ Bills,12 - 21,33.0,Bills,Home,Jets,12.0,9.0,...,42.0,0.39,0.61,Under,Over,9.0,Bills,Under,Minority,Majority
2,1,2017-09-10,Falcons @ Bears,23 - 17,40.0,Falcons,Away,Falcons,23.0,-6.0,...,48.0,0.51,0.49,Over,Under,6.0,Bears,Under,Minority,Minority
3,1,2017-09-10,Ravens @ Bengals,20 - 0,20.0,Ravens,Away,Ravens,20.0,-20.0,...,41.5,0.47,0.53,Under,Over,20.0,Ravens,Under,Minority,Majority
4,1,2017-09-10,Steelers @ Browns,21 - 18,39.0,Steelers,Away,Steelers,21.0,-3.0,...,47.0,0.39,0.61,Under,Over,3.0,Browns,Under,Minority,Majority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,17,2017-12-31,Bills @ Dolphins,22 - 16,38.0,Bills,Away,Bills,22.0,-6.0,...,41.5,0.55,0.45,Over,Under,6.0,Bills,Under,Majority,Minority
237,17,2017-12-31,Raiders @ Chargers,10 - 30,40.0,Chargers,Home,Raiders,10.0,20.0,...,43.0,0.65,0.35,Over,Under,20.0,Chargers,Under,Push,Minority
238,17,2017-12-31,Cardinals @ Seahawks,26 - 24,50.0,Cardinals,Away,Cardinals,26.0,-2.0,...,38.0,0.59,0.41,Over,Under,2.0,Cardinals,Over,Majority,Majority
239,17,2017-12-31,Saints @ Buccaneers,24 - 31,55.0,Buccaneers,Home,Saints,24.0,7.0,...,49.0,0.45,0.55,Under,Over,7.0,Buccaneers,Over,Minority,Minority


In [58]:
final_df_nfl_2017.to_csv('final_df_nfl_2017.csv')